In [1]:
import os
import ast
import json
import random
import pandas as pd

from raw.generation_util.parsing import pqt_extract_ground_truth

In [16]:
task_split = "train"
task_type = "predictmove"

parent_dir = f"cleaned/verl_tasks/{task_split}"
filename = f"{task_type}.parquet"
filepath = os.path.join(parent_dir, filename)

# Load data agnostic to parquet / json / jsonl
if filename.endswith('.parquet'):
    df = pd.read_parquet(filepath)
    data = df.to_dict('records')
    print(f"Loaded {len(data)} items from parquet file {filename}.")
elif filename.endswith('.json'):
    with open(filepath, 'r') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} items from JSON file {filename}.")
elif filename.endswith('.jsonl'):
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    print(f"Loaded {len(data)} items from JSONL file {filename}.")
else:
    raise ValueError(f"Unsupported file format: {filename}. Please use .parquet, .json, or .jsonl files.")

random.shuffle(data)

# Need to coerce since we stringified the ground truth dict due to issues w/ parquet saving
for row in data:
    row['reward_model']['ground_truth'] = pqt_extract_ground_truth(row['reward_model']['ground_truth'], task_type = task_type)

print(f"Shuffled {len(data)} items.")

Loaded 1024 items from parquet file predictmove.parquet.
Shuffled 1024 items.


In [17]:
# Randomly sample / print data
sample = random.choice(data)
# for key, value in sample.items():
#     print(key)
#     print(type(value))
#     print(f"{value}\n{'-'*60}\n")

sys_prompt  = sample['prompt'][0]
user_prompt = sample['prompt'][1]
ground_truth = sample['reward_model']['ground_truth']

print(f"System:\n{sys_prompt['content']}\n")
print(f"User:\n{user_prompt['content']}\n")
print(f"Ground Truth:\n{ground_truth}")

System:
You are a chess grandmaster currently playing against a strong opponent. Assume they will capitalize on any blunders you make.

You are exceptional at thinking through various board states and strategically planning your next move. You are detail oriented, strategic, and an efficient thinker. You use all of these traits to be an effective chess player.

Do not hallucinate. All statements should be based on the provided board and you must avoid considering pieces that do not exist or moves that are illegal.

You must refer to moves in UCI notation (e.g., d7d5) and include your thinking in think tags (e.g., <think> your_thinking </think>) and your answer in answer tags (e.g., <answer> your_answer </answer>). If you do not need to reason heavily, you should still include think tags but with '\n\n' (e.g., <think> \n\n <\think>).

Finally, make sure to always think out loud (using the <think> tags) to convey your thought process as you consider various moves and solutions to puzzles

## Code to clean up / concatenate existing data

In [4]:
# # Example usage:
# parent_dir = "explainer_data"
# filenames = [
#     "explainer_clean_100_1558_15.parquet",
#     "explainer_clean_1250.parquet",
#     "explanations_0_1000_0104_16.parquet",
#     "explanations_1_1000_0330_16.parquet",
#     "explanations_2_1000_0557_16.parquet",
#     "explanations_3_1000_0826_16.parquet"
# ]
# filepaths = [os.path.join(parent_dir, fn) for fn in filenames]
# output_file = "combined_chessexplainer.jsonl"
# system_prompt_file = "chess_task_sysprompt.txt"  # Or "llama4_default_sysprompt.txt"


# HEADER_PATTERN = re.compile(
#     r"<\|start_header_id\|>(\w+)<\|end_header_id\|>\n?(.*?)(?=(<\|start_header_id\|>|\Z|<\|eot_id\|>))",
#     re.DOTALL
# )

# def extract_dialogue(sample):
#     text = sample['prompt']
#     result = []
#     for match in HEADER_PATTERN.finditer(text):
#         role, content = match.group(1), match.group(2).strip()
#         # Remove Llama tags and both eot_id types
#         content = re.sub(r"<\|.*?\|>|<eot_id>", "", content).strip()
#         if role == "system":
#             content = "chess_task_sysprompt.txt"
#         if role == "user":
#             prefix = "Here is a board in a game you're currently playing. I want you to think through some possible moves you could make and how those moves will likely play out. You may find it helpful to roll-out each line assuming your opponent plays near-optimally. You may also find it helpful to consider the value of the final board state after each roll-out.\n\nAfter you think through your various moves, please end by telling me your chosen move (in UCI notation) within answer tags.\n\n"
#             content = prefix + content
#         if content:
#             result.append((role, content))
#     completion = sample.get('completion', '').strip()
#     completion = re.sub(r'(<\|eot_id\|>|<eot_id>)\s*$', '', completion).strip()
#     if completion:
#         result.append(('assistant', completion))
#     return result

# def convert_and_save(parquet_paths, output_path):
#     all_dialogues = []
#     for path in parquet_paths:
#         df = pd.read_parquet(path)
#         all_dialogues.extend(
#             {"chat": extract_dialogue(row)} for row in df.to_dict('records')
#         )
#     # Save as JSONL
#     with open(output_path, 'w', encoding='utf-8') as f:
#         for d in all_dialogues:
#             json.dump(d, f, ensure_ascii=False)
#             f.write('\n')
#     print(f"Saved {len(all_dialogues)} dialogues to {output_path}")


# output_path = os.path.join(parent_dir, output_file)
# convert_and_save(filepaths, output_path)